In [13]:
import pandas as pd
import numpy as np
import vowpalwabbit
vw = vowpalwabbit.Workspace("--cb 4", quiet=True)


In [14]:
train_data = [
    {
        "action": 1,
        "cost": 2,
        "probability": 0.4,
        "feature1": "a",
        "feature2": "c",
        "feature3": "",
    },
    {
        "action": 3,
        "cost": 0,
        "probability": 0.2,
        "feature1": "b",
        "feature2": "d",
        "feature3": "",
    },
    {
        "action": 4,
        "cost": 1,
        "probability": 0.5,
        "feature1": "a",
        "feature2": "b",
        "feature3": "",
    },
    {
        "action": 2,
        "cost": 1,
        "probability": 0.3,
        "feature1": "a",
        "feature2": "b",
        "feature3": "c",
    },
    {
        "action": 3,
        "cost": 1,
        "probability": 0.7,
        "feature1": "a",
        "feature2": "d",
        "feature3": "",
    },
]

train_df = pd.DataFrame(train_data)

train_df["index"] = range(1, len(train_df) + 1)
train_df = train_df.set_index("index")

In [ ]:

test_data = [
    {"feature1": "b", "feature2": "c", "feature3": ""},
    {"feature1": "a", "feature2": "", "feature3": "b"},
    {"feature1": "b", "feature2": "b", "feature3": ""},
    {"feature1": "a", "feature2": "", "feature3": "b"},
]

test_df = pd.DataFrame(test_data)

test_df["index"] = range(1, len(test_df) + 1)
test_df = test_df.set_index("index")


In [16]:
train_df.head()


,action,cost,probability,feature1,feature2,feature3
index,,,,,,
1,1,2,0.4,a,c,
2,3,0,0.2,b,d,
3,4,1,0.5,a,b,
4,2,1,0.3,a,b,c
5,3,1,0.7,a,d,


In [17]:
for i in train_df.index:
    action = train_df.loc[i, "action"]
    cost = train_df.loc[i, "cost"]
    probability = train_df.loc[i, "probability"]
    feature1 = train_df.loc[i, "feature1"]
    feature2 = train_df.loc[i, "feature2"]
    feature3 = train_df.loc[i, "feature3"]

    # Construct the example in the required vw format.
    learn_example = (
        str(action)
        + ":"
        + str(cost)
        + ":"
        + str(probability)
        + " | "
        + str(feature1)
        + " "
        + str(feature2)
        + " "
        + str(feature3)
    )

    # Here we do the actual learning.
    vw.learn(learn_example)

In [18]:
for j in test_df.index:
    feature1 = test_df.loc[j, "feature1"]
    feature2 = test_df.loc[j, "feature2"]
    feature3 = test_df.loc[j, "feature3"]

    test_example = "| " + str(feature1) + " " + str(feature2) + " " + str(feature3)

    choice = vw.predict(test_example)
    print(j, choice)

1 3
2 3
3 3
4 3


In [19]:
vw.save("cb.model")
del vw

vw = vowpalwabbit.Workspace("--cb 4 -i cb.model", quiet=True)
print(vw.predict("| a b"))


3
